01_make_pairs_and_patches

This notebook makes a simple train/val patch dataset as .npz files.
If you already have real maps and labels, replace the dummy data block with your loader.


In [ ]:
import numpy as np
from pathlib import Path

from unetloopdetection.io import save_npz
from unetloopdetection.patches import PatchSpec, extract_pair_patches

rng = np.random.default_rng(0)
out_dir = Path("../data")
out_dir.mkdir(parents=True, exist_ok=True)

# dummy example: a contact-map-like image + a binary mask
H, W = 256, 256
x = rng.random((H, W), dtype=np.float32)

# make a few synthetic 'loops' as circles in the mask
yy, xx = np.ogrid[:H, :W]
mask = np.zeros((H, W), dtype=np.uint8)
for (cy, cx, r) in [(64, 80, 10), (160, 180, 12), (120, 140, 8)]:
    mask |= ((yy - cy)**2 + (xx - cx)**2) <= r**2

spec = PatchSpec(size=64, stride=32, pad=True)
px, py, coords = extract_pair_patches(x, mask, spec)

print(px.shape, py.shape, len(coords))

In [ ]:
# split train/val and save
N = px.shape[0]
idx = np.arange(N)
rng.shuffle(idx)

n_train = int(0.8 * N)
tr = idx[:n_train]
va = idx[n_train:]

save_npz(out_dir / "train_patches.npz", X=px[tr], y=py[tr])
save_npz(out_dir / "val_patches.npz", X=px[va], y=py[va])

print("saved:", out_dir / "train_patches.npz", out_dir / "val_patches.npz")